In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import os
import cv2
import GPUtil
import matplotlib.pyplot as plt                        
%matplotlib inline  

# In a multi GPU server, chose which to use:
NUMBER_OF_GPUS_TO_USE = 1
Availability=GPUtil.getAvailability(GPUtil.getGPUs())
all_gpus = np.arange(3)
available_gpu_indexes = [x for x in all_gpus if Availability[x]]
# Set CUDA_DEVICE_ORDER so the IDs assigned by CUDA match those from nvidia-smi
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# Set CUDA_VISIBLE_DEVICES to mask out all other GPUs than the first NUMBER_OF_GPUS_TO_USE available device id
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(np.array(available_gpu_indexes[:NUMBER_OF_GPUS_TO_USE]).astype(str))

import tensorflow as tf
config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

import keras
from keras.backend.tensorflow_backend import set_session

set_session(tf.Session(config=config))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image                  
from tqdm import tqdm

from keras.models import Sequential
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.core import Dense

from keras.applications.resnet50 import preprocess_input, decode_predictions

from sklearn.datasets import load_files       
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from glob import glob

from keras.engine.input_layer import Input
from keras.models import Model

from keras.callbacks import ModelCheckpoint

In [6]:
import string, random
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

### Check if its a human face

In [2]:
# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

### Confirm the image is actually a dog

The ability of this process is limited by the dataset

In [3]:
# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

In [4]:
def path_to_tensor(img_path, expand=True):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    if expand:
        return np.expand_dims(x, axis=0)
    else:
        return x

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [5]:
def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

### Use transfer learning to reduce training time

Model architecture

In [7]:
### Define architecture.
Resnet50_model = Sequential()
Resnet50_model.add(GlobalAveragePooling2D(input_shape=(7, 7, 2048)))
Resnet50_model.add(Dense(120, activation='softmax'))

In [8]:
Resnet50_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               245880    
Total params: 245,880
Trainable params: 245,880
Non-trainable params: 0
_________________________________________________________________


#### Compile and test

##### Load data

In [10]:
data = load_files("/notebooks/Practice/imgnet-dogbreed/Images/")

In [12]:
train_files, test_files, train_targets, test_targets = train_test_split(data['filenames'], data['target'], test_size=0.2)
test_files, validation_files, test_targets, validation_targets = train_test_split(test_files, test_targets, test_size=0.5)
train_targets = np_utils.to_categorical(np.array(train_targets), 120)
test_targets = np_utils.to_categorical(np.array(test_targets), 120)
validation_targets = np_utils.to_categorical(np.array(validation_targets), 120)

dog_names = [item.split('/')[-1] for item in sorted(glob("/notebooks/Practice/imgnet-dogbreed/Images/*"))]

In [16]:
# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, validation_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d test dog images.'% len(validation_files))
print('There are %d test dog images.'% len(test_files))

There are 120 total dog categories.
There are 20580 total dog images.

There are 16464 training dog images.
There are 2058 test dog images.
There are 2058 test dog images.


Create bottleneck features

In [3]:
train_features = np.load('ResNet50/train_features.npy')
test_features = np.load('ResNet50/test_features.npy')
validation_features = np.load('ResNet50/validation_features.npy')

train_targets = np.load('ResNet50/train_targets.npy')
test_targets = np.load('ResNet50/test_targets.npy')
validation_targets = np.load('ResNet50/validation_targets.npy')

train_files = np.load('ResNet50/train_files.npy')
test_files = np.load('ResNet50/test_files.npy')
validation_files = np.load('ResNet50/validation_files.npy')

In [163]:
input_img = Input(shape=(224, 224, 3))

model_resnet = ResNet50(weights='imagenet', include_top=False,
    input_shape=(224, 224, 3),
    input_tensor=input_img)

In [164]:
%time train_features = model_resnet.predict(preprocess_input(paths_to_tensor(train_files)))

CPU times: user 2min 47s, sys: 28.8 s, total: 3min 15s
Wall time: 3min 9s


In [165]:
%time test_features = model_resnet.predict(preprocess_input(paths_to_tensor(test_files)))

CPU times: user 20.4 s, sys: 3.01 s, total: 23.4 s
Wall time: 22.6 s


In [166]:
%time validation_features = model_resnet.predict(preprocess_input(paths_to_tensor(validation_files)))

CPU times: user 20.4 s, sys: 2.94 s, total: 23.3 s
Wall time: 22.5 s


In [167]:
[[np.shape(train_features), np.shape(test_features), np.shape(validation_features)],
 [np.shape(train_targets), np.shape(test_targets), np.shape(validation_targets)]]

[[(16464, 7, 7, 2048), (2058, 7, 7, 2048), (2058, 7, 7, 2048)],
 [(16464, 120), (2058, 120), (2058, 120)]]

In [170]:
np.save('ResNet50/train_features', train_features)
np.save('ResNet50/test_features', test_features)
np.save('ResNet50/validation_features', validation_features)
np.save('ResNet50/train_targets', train_targets)
np.save('ResNet50/test_targets', test_targets)
np.save('ResNet50/validation_targets', validation_targets)

np.save('ResNet50/train_files',train_files)
np.save('ResNet50/test_files',test_files)
np.save('ResNet50/validation_files',validation_files)

In [26]:
### Compile the model.
Resnet50_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [48]:
Resnet50_model.compile(optimizer=keras.optimizers.Nadam(),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=[keras.metrics.categorical_accuracy])

In [52]:
### Train the model.
checkpointer_resnet = ModelCheckpoint(filepath='ResNet50/weights.best.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

tb = keras.callbacks.TensorBoard(log_dir='./logs'+'/'+id_generator(), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

In [53]:
Resnet50_model.fit(train_features, train_targets, 
          validation_data=(validation_features, validation_targets),
          epochs=50, batch_size=50, callbacks=[checkpointer_resnet, tb], verbose = 0)


Epoch 00001: val_loss improved from inf to 2.97892, saving model to ResNet50/weights.best.ResNet50.hdf5

Epoch 00002: val_loss improved from 2.97892 to 2.96905, saving model to ResNet50/weights.best.ResNet50.hdf5

Epoch 00003: val_loss improved from 2.96905 to 2.91374, saving model to ResNet50/weights.best.ResNet50.hdf5

Epoch 00004: val_loss improved from 2.91374 to 2.89610, saving model to ResNet50/weights.best.ResNet50.hdf5

Epoch 00005: val_loss did not improve from 2.89610

Epoch 00006: val_loss did not improve from 2.89610

Epoch 00007: val_loss improved from 2.89610 to 2.79224, saving model to ResNet50/weights.best.ResNet50.hdf5

Epoch 00008: val_loss did not improve from 2.79224

Epoch 00009: val_loss did not improve from 2.79224

Epoch 00010: val_loss did not improve from 2.79224

Epoch 00011: val_loss did not improve from 2.79224

Epoch 00012: val_loss did not improve from 2.79224

Epoch 00013: val_loss did not improve from 2.79224

Epoch 00014: val_loss did not improve from

KeyboardInterrupt: 

In [9]:
### Load the model weights with the best validation loss.
Resnet50_model.load_weights('ResNet50/weights.best.ResNet50.hdf5')

In [54]:
### Calculate classification accuracy on the test dataset.
predictions = np.argmax(Resnet50_model.predict(test_features), axis=1)
actual = np.argmax(test_targets, axis=1)
# Report test accuracy
test_accuracy = 100*np.sum(predictions==actual)/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 74.8299%


Test accuracy: 75.6074%
